**Ch10: 爬虫基础**

Source: 
[数据分析实战 45 讲](https://time.geekbang.org/column/intro/100021701?tab=intro)

---

## 爬虫前的背景知识

三月份记的，真是太小白了 -_-）

### HTTP (HyperText Transfer Protocol)

用于在网络上传输超文本(如HTML）的protocol，是客户端（如浏览器）和服务器之间通信的基础。Python上解析它用到request库，分`get()`, `post()`，后者用于向服务器提交数据，比如文件、表单。

`request.get()`：

- status_code：200: successfull; 404: not found; 403: forbidden: 500: server error

- headers: 伪装这个爬虫是个浏览器

    - content-type: 用于指示返回内容的类型。
 
    - `print(response.headers['Content-Type'])`此时为API接口，返回json数据。直接供程序使用，无需渲染页面。 
               
    - 常见content-type有：text/html：HTML页面；application/json：JSON数据；image/png。

### 正在使用markdown，那么HTML和XML这些markup标记语言是什么？？

- 标记语言是一种用成对标签（`<tag>`和`</tag>`，也就是HTML中说的**node/节点**）描述文档结构和内容的语言。

- 而Markdown是一种轻量级语言，用简单的符号（如 `#`、`*`）格式化文本。

### HTML (HyperText Markup Langugae) 

一种用于创建网页的语言啦。网页的骨架，有一系列标签。

**即HTTP负责将HTML文件从服务器传输到客户端（如浏览器）**；HTML是内容格式，定义了网页的结构和内容，浏览器解析HTML并渲染成用户看到的页面。

XPath是用于在HTML文档中提取元素的选择器，又有它自己的一套语言。而BeautifulSoup提供类似Python的链式方法（eg. `find()`, `select()`），不像XPath那样有自己的表达式，也可以用Regex。

---

##  爬虫流程

### 打开网页

用requests库发送HTTP请求。也可以Selenium用于自动化浏览器操作（如处理动态加载的页面），但这个开销大，一般先用requests快速获取数据，若遇到动态内容（如JavaScript渲染的那种网页，那种打开源码也找不着的必须Cmd+Shift+C 'Inspect element'的），再改用Selenium。但是Selenium也可以自动化点击等等！很好玩喵，后面某章会讲到。

### 提取并保存数据

- 解析器，比如BeautifulSoup方法中的'html.parser'参数及lxml可将request打开之后的HTML文档解析成Python对象

- 选择器，在HTML或XML这样子的文档里定位并提取想找的元素，比如HTML需要用XPath或者CSS选择器。用BeautifulSoup提取元素的话可以用Regex进行**字符串匹配**，需导入re库。如果是json数据，用json库解析成Python对象：`json.load()`；`json.dumps()`转换回来。

- 保存至csv/xls。

---

## XPath Syntax

```
//tagname[@attribute = ‘value’]
```

注意！！它**不像regex一样是匹配夹在两边之间的**，而是以**节点内部的**（从前一个`<`到相匹配的`>`结束）选择，除非`text()`。

* /：从根节点开始（绝对路径）。
  
* //：从**任意**位置匹配（相对路径）。相对路径想要增加父节点作为约束也是用俩斜杠，eg. `//div[@class="title"]//a[@class="title-text"]`

* .：当前节点; ..：父节点。

* @：选择attribute，`<tagname attribute="value">`。

    * `//div[contains(@id, 'something')]` 取`<div id="...something...">`的节点。

    * `//div[starts-with(@id, 'something')]` 取`<div id="something...">`的节点。
 
* \*：匹配任意(全部）节点。

* \@*：匹配任意(全部）属性。

* []：**用于过滤节点**（不然怎么知道相对路径匹配到的是哪一个）

    * `//div[@*]`选择所有带有属性的div
    
    * `//div[1]`选择第一个div
    
    * `//div[last()]`选择最后一个div
    
    * `//div[position()<3]`选择前两个div
    
    * `//div[@class="box"]`选择class属性为"box"的div
    
    * `//p/text()`提取所有成对`<p>`之间的文本。

#### 重点表格！！

| XPath expression | What it matches | Example output (for `<a href="/song?id=123">Link</a>`) |
|------------------|-----------------|--------------------------------------------------------|
| `//a`            | All `<a>` element nodes in the document | `<a href="/song?id=123">Link</a>` |
| `//a/@href`      | All `href` attribute **values** of `<a>` elements | `"/song?id=123"` (*just the URL string*) |
| `//a/text()`     | All **text content** inside `<a>` elements | `"Link"` |


---

## Regex 及其在HTML中用法

- Scrutinize the actual source code.

- Use raw strings (`r'pattern'`)，要不然有的python字符还得转义，加上这个就只用转义regex的特殊字符（下面有列举）。

### Basic re.functions()

- `re.findall()` - Returns all matches as a list;

- `re.search()` - Returns a match object for the first match;

- `re.match()` - Checks only at string beginning;

- `re.sub()` - Replaces matches.

### Common patterns:

* `.*` - Matches any character *(except newlines eg. `\n`/`\r`, but includes whitespaces)* 0+ times. `+` - 1+ times; `?` - 0/1 times (optional); {n} - exactly n times (eg. `\d{3}` -> "123"); {n,} - n or more times.

    * `.*?` - Non-greedy version (matches **as little as possible**)

* `\d` - Digit, **`\s` - whitespace** (space, tab, newline), `\w` - word character (a-z, A-Z, 0-9, _).

* `\D` - Non-digit, `\S` - non-whitespace, `\W` - non-word character (eg. '@').

* `^` - Start of string: `^\d` -> "1 apple"; `$` - end of string: `\d$` -> "apple 2".

* `\[^ ]` Any character **not** in brackets: `[^\d]` -> "a", "-"

* `( )` Captures group: (\d+)-(\d+) -> "123-456" -> captured as groups ("123", "456").


**HTML elements frequently contain:**

* Line breaks (`\n`) or returns (`\r`)

* Tabs (`\t`)

* Regular spaces ( )

* Non-breaking spaces (`&nbsp;`) 

**Special characters that you must escape (add `\`) in regex:**
`^ $ * + ? { } [ ] \ | ( )`

*  Inside character classes [], most metacharacters lose special meaning and don't need escaping:

Even after removing `&nbsp;`, residual whitespace often remains. So: use **strip()** to remove whitespaces.

* HTML中类似的特殊符号：

```
&amp; → &
&lt; → < 
&gt; → > 
&quot; → " 
&copy; → ©
```

* `<br>`: 换行，相当于回车。像`<br>`和`<img>`这样的标签是void elements，在HTML中不需要关闭标签 。


**Attribute matching (like 'href') is more reliable than tag content matching** because:

- Attributes follow strict `name="value"` patterns.

- Less variation in how they're written.
  
- Account for possible whitespace (\s*) in tag content (eg. `<p class="quote">\s*<span>(.*?)</span>\s*</p>)`

`<[^>]+>`: matching HTML tag since `[^>]+` matches 1+ characters that are **NOT '>'** (using ^ negation)

用BeautifulSoup时可以直接用它的函数，到了模糊定位的时候再用Regex：

In [7]:
from bs4 import BeautifulSoup
html = """
<div class="content">
    <h1>主标题</h1>
    <a href="/page1">链接1</a>
    <a href="/page2">链接2</a>
</div>"""

soup = BeautifulSoup(html, 'html.parser')
# 提取第一个h1标签的文本，好方便。find()返回第一个匹配的元素, find_all()返回所有匹配的元素组的列表。
title = soup.find('h1').text
print(title)
# 提取所有a标签的href属性
links = [a['href'] for a in soup.find_all('a')]
print(links) 
# 选择所有class="content"的div；select()是使用CSS选择器语法定位元素
contents = soup.select('div.content')
print([content.text for content in contents]) 

主标题
['/page1', '/page2']
['\n主标题\n链接1\n链接2\n']


In [13]:
# soup + regex版:
import os
import urllib.request
from bs4 import BeautifulSoup
import re
import xlwt

def main():
    url = "https://movie.douban.com/top250?start="
    # 在网页上直接右键show page source就可以看到html，然后按照它写正则表达式匹配
    datalist = getData(url) # 这个存储遍历结果
    savepath = "豆瓣电影 爬虫结果Top100.xls"
    saveData(datalist,savepath)

def getData(url):
    datalist = [] # 这个放在遍历外面；而for循环内部的data[]会自身不断更新但每次都添加进datalist[]
    for i in range(4):
        pageurl = url + str(i) # 页面上每次只有25条，所以要4次
        response = askURL(pageurl)
        soup = BeautifulSoup(response, 'html.parser')
        for item in soup.find_all('div', attrs={'class': 'item'}):
                # name – A filter on **tag** name.
                # attrs – A dictionary of filters on attribute values.
                # recursive – If True, find_all() will perform a recursive search of this PageElement's children. Otherwise, only the direct children will be considered.
                # limit – Stop looking after finding this many results.
            data = []
            item = str(item)
            link = re.findall(r'href="(.*?)"', item)[0]  # regex grouping，加括号之后缓存成一个对象/列表
            data.append(link)
            title = re.findall(r'<span class="title">(.*?)</span>', item) # 一定跟源码里有没有>对应好了；
            data.append(title[0])  # 中文标题
            data.append(title[1].replace('&nbsp;/&nbsp;', '').replace('\xa0/', '') # &nbsp; becomes ‘\xa0’ in Python...?
                .strip()) if len(title) > 1 else ''
            rating = re.findall(r'<span class="rating_num" property="v:average">(.*?)</span>', item)[0]
            data.append(rating)
            numrated = re.findall(r'<span>(\d*)人评价</span>', item)[0]
            data.append(numrated)
            # 提取引言（修正为匹配<p class="quote"><span>...</span></p>结构）
            quote_match = re.findall(r'<p class="quote">\s*<span>(.*?)</span>\s*</p>', item, re.DOTALL)
            quote = quote_match[0].strip() if quote_match else " "
            data.append(quote)
            bd_match = re.findall(r'<div class="bd">\s*<p>(.*?)</p>', item, re.DOTALL) #without DOTALL: fail on multi-line content
            if bd_match:   # 天天报错，可能是没匹配出来
                bd = re.sub(r'<[^>]+', " ", bd_match[0])
                    # Return the *string* obtained by replacing the *leftmost* non-overlapping occurrences of the pattern in string by the replacement on the right
                bd = re.sub(r'\s+', " ", bd)
                data.append(bd.strip())
            else:
                raise ValueError("赶紧停这吧别跑下去了")
            datalist.append(data)
    return datalist

def askURL(pageurl):
    request = urllib.request.Request(pageurl, headers={'User-Agent': 'Safari / 537.36'})
    html = "" 
    response = urllib.request.urlopen(request)
    response = response.read().decode('utf-8')
    return response

def saveData(datalist,savepath):
    print("Saving————")
    workbook = xlwt.Workbook(encoding='utf-8',style_compression=0)
    sheet = workbook.add_sheet('豆瓣电影 爬虫结果Top100',cell_overwrite_ok=True)
    col = ("link","title","rating","ppl rated", "quote", "additional info")
    for i in range(6): 
        sheet.write(0,i,col[i]) #第一行第i列，这是遍历写入列名
    for i in range(100):
        data2 = datalist[i]
        for j in range(len(data2)):
            sheet.write(i+1,j,data2[j]) #行是i+1是因为第一行被列名占了
            # datalist是个二维数组，data2及之前的data都是一位数组（回忆咋append进去的）
    workbook.save(savepath)


if __name__ == '__main__':
    main()
    print("喵喵喵喵喵喵恭喜！！！！")



Saving————
喵喵喵喵喵喵恭喜！！！！


In [ ]:
# 如果用XPath的话：（不用soup了，但还是用request爬取静态页面）：
from lxml import etree
import csv
from time import sleep

def scrape_douban_top100():
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    base_url = "https://movie.douban.com/top250"
    data = []
    for start in range(0, 100, 25):  # 一样还是request爬取
        url = f"{base_url}?start={start}"
        response = requests.get(url, headers=headers)
        # 如果不用request的话也可以这样：
        # driver = webdriver.Chrome()
        
        html = etree.HTML(response.text)
        movies = html.xpath('//div[@class="item"]')
        for movie in movies:
            
       # soup是这样的：（这里写法差不多，只是下文匹配文本方式不同）
            # soup = BeautifulSoup(response, 'html.parser')
            # for item in soup.find_all('div', attrs={'class': 'item'}):
            
            item = {}
            item['title'] = movie.xpath('.//span[@class="title"][1]/text()')[0]
            # ...
            item['link'] = movie.xpath('.//div[@class="hd"]/a/@href')[0]
            
            data.append(item)
            # 字典作为单个元素存在于列表中！
            # data = [
            #    {"title": "电影1", "rating": 9.0, ...},  第1个字典, i.e. data[0]
            #    {"title": "电影2", "rating": 8.5, ...},  第2个字典
            #     ...] 其他字典
            # 保留字典结构便于后续通过键名访问字段,如直接data[0]["title"]
        
        sleep(2)  
        
    # 保存为CSV
    with open('douban_top100.csv', 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=data[0].keys()) # 注意注意这个.keys()，这是因为列表data中的每个元素自己都是一个字典哇
        # csv.DictWriter：专门用于将字典数据写入CSV
        # `fieldnames=data[0].keys()`：将第一个字典的键（如 title, rating等）作为CSV的列名
        writer.writeheader()
        # 将 fieldnames 中定义的列名写入CSV文件的第一行。
        writer.writerows(data)
        # 每个字典对应CSV中的一行，字典的键与fieldnames列名自动匹配。
        # 如果某字典缺少fieldnames中的键，对应单元格会留空。如果字典有多余的键，会被忽略。
    print("数据已保存到 douban_top100.csv")
if __name__ == '__main__':
    scrape_douban_top100()